In [12]:
import random
import itertools
import pandas as pd
from alive_progress import alive_bar

In [ ]:
def generate_random_play(deck, n_players, min_cards = 1, max_cards = 5):

    n_cards = random.randint(n_players*min_cards, n_players*max_cards)
    play = random.sample(deck, n_cards)

    return n_cards, n_players, play


def find_combinations(play):

    combs = {
        "one pair":         0,
        "two pairs":        0,
        "three of a kind":  0,
        "straight":         0,
        "flush":            0,
        "full house":       0,
        "four of a kind":   0,
        "straight flush":   0,
    }

    play.sort(key = lambda x: x[0])

    combs["one pair"]        = find_one_pair(play, is_sorted=True)
    combs["two pairs"]       = find_two_pairs(play, is_sorted=True)
    combs["three of a kind"] = find_three_of_a_kind(play, is_sorted=True)
    combs["four of a kind"]  = find_four_of_a_kind(play, is_sorted=True)
    combs["full house"]      = find_full_house(play, is_sorted=True)
    combs["straight"]        = find_straight(play)
    combs["flush"]           = find_flush(play)
    combs["straight flush"]  = find_straight_flush(play)

    return combs


def find_one_pair(play, is_sorted=False):

    if not is_sorted: play.sort(key = lambda x: x[0])
    
    for i in range(1, len(play)):
        if play[i-1][0] == play[i][0]: return 1

    return 0       


def find_two_pairs(play, is_sorted=False):

    if len(play) < 4: return 0

    if not is_sorted: play.sort(key = lambda x: x[0])

    pairs = 0

    for _, group in itertools.groupby(play, lambda x: x[0]): 
        if len(list(group)) > 1: pairs += 1

    return 1 if pairs > 1 else 0


def find_three_of_a_kind(play, is_sorted=False):

    if len(play) < 3: return 0

    if not is_sorted: play.sort(key = lambda x: x[0])

    for i in range(2, len(play)):
        if play[i-2][0] == play[i][0]: return 1

    return 0


def find_four_of_a_kind(play, is_sorted=False):

    if len(play) < 4: return 0

    if not is_sorted: play.sort(key = lambda x: x[0])
    
    for i in range(3, len(play)):
        if play[i-3][0] == play[i][0]: return 1

    return 0


def find_full_house(play, is_sorted=False):

    if len(play) < 5: return 0

    if not is_sorted: play.sort(key = lambda x: x[0])

    groups = []
    
    for _, group in itertools.groupby(play, lambda x: x[0]): 
        groups.append(len(list(group)))

    groups.sort(reverse=True)

    if groups[0] > 2 and groups[1] > 1: return 1

    return 0


def find_straight(play):

    play = list(set(c[0] for c in play))
    play.sort()
    
    if len(play) < 5: return 0

    for i in range(len(play)):
        if play[i-4] == play[i] - 4: return 1

    return 0


def find_flush(play):

    if len(play) < 5: return 0

    play.sort(key = lambda x: x[1])

    for _, group in itertools.groupby(play, lambda x: x[1]): 
        if len(list(group)) > 4: return 1

    return 0


def find_straight_flush(play):

    if len(play) < 5: return 0

    play.sort(key = lambda x: x[1])

    for _, group in itertools.groupby(play, lambda x: x[1]): 
        subplay = list(group)
        if find_straight(subplay) == 1: return 1

    return 0

In [19]:
values = range(5, 15)
suits = "HDCS"

deck = [(v, s) for v in values for s in suits]

active_players = [2, 3, 4, 5, 6]

df = pd.DataFrame(columns=[
    "active_players",
    "active_cards",
    "one pair",
    "two pairs",
    "three of a kind",
    "straight",
    "flush",
    "full house",
    "four of a kind",
    "straight flush"])

In [ ]:
iterations_per_player = 100000

with alive_bar(sum(active_players)*iterations_per_player, force_tty = True) as bar:
        
    for i, ap in enumerate(active_players):

        for _ in range(ap*iterations_per_player):
            n_cards, n_players, play = generate_random_play(deck, ap)
            results = find_combinations(play)

            results.update({
                "active_players": n_players,
                "active_cards": n_cards,
            })

            df.loc[len(df)] = results

            bar()

|█▋                                      | ▆█▆ 80971/2000000 [4%] in 4:10 (~1:38

In [ ]:
df.groupby("active_players").mean().to_csv("results.csv")

,active_cards,one pair,two pairs,three of a kind,straight,flush,full house,four of a kind,straight flush
active_players,,,,,,,,,
2,6.230000,0.730000,0.350000,0.150000,0.130000,0.090000,0.070000,0.000000,0.000000
3,9.266667,0.846667,0.646667,0.373333,0.366667,0.346667,0.293333,0.073333,0.013333
4,12.060000,0.925000,0.775000,0.575000,0.550000,0.560000,0.465000,0.135000,0.100000
5,15.624000,0.972000,0.884000,0.748000,0.696000,0.732000,0.684000,0.240000,0.228000
6,17.743333,0.976667,0.880000,0.780000,0.750000,0.760000,0.740000,0.406667,0.350000
